# Laboratorio 3

Si richiede un’implementazione della teoria sulle valenze di Patrick Hanks. In particolare, partendo da un corpus a scelta e uno specifico verbo (tendenzialmente non troppo frequente e/o generico ma nemmeno raro), l’idea è di costruire dei possibili cluster semantici, con relativa frequenza. Ad es. dato il verbo "to see" con valenza = 2, e usando un parser sintattico (ad es. Spacy), si possono collezionare eventuali fillers per i ruoli di subj e obj del verbo, per poi convertirli in semantic types. Un cluster frequente su "to see" potrebbe unire subj = noun.person con obj = noun.artifact. Si richiede di partire da un corpus di almeno alcune centinaia di istanze del verbo.

In [1]:
import pandas as pd
import spacy

In [67]:
parser = spacy.load('en_core_web_sm')
verbs = {}

i = 0
for sentence in corpus['sentence']:
    sentence = sentence.replace('<s>', '').replace('</s>', '').replace('"', '').replace("'", '')[1:]
    parsified_sentence = parser(sentence)
    #spacy.displacy.render(parsified_sentence, style="dep", options={"compact": True}) # visualizza il grafo


    verb_deps = {}

    # per ogni verbo della frase prendo tutte le dipendenze di quel verbo
    for token in parsified_sentence:
        if token.head.pos_ == 'VERB':
            if token.head.lemma_ not in verb_deps:
                verb_deps[token.head.lemma_] = set()

            if token.dep_ != 'ROOT' and token.dep_ != 'punct':
                verb_deps[token.head.lemma_].add(token.dep_)

    # maniglia al verbo 
    current_keys = {}

    # controllo se esiste già un verbo con le stesse dipendenze altrimenti lo creo
    for verb, deps in verb_deps.items():
        current_keys[verb] = None

        # prelevo le chiavi che matchano con il verbo corrente (es. "see_1" matcha con "see")
        matching_keys = [key for key in verbs.keys() if verb in key]


        # controllo se esiste già un verbo con le stesse dipendenze
        if matching_keys:
            for key in matching_keys:
                if deps == set(verbs[key].keys()):
                    current_keys[verb] = key

        # se non esiste un verbo con le stesse dipendenze lo creo
        if current_keys[verb] is None:
            current_keys[verb] = verb + "_" + str(i)
            verbs[current_keys[verb]] = {}
            for dep in deps:
                verbs[current_keys[verb]][dep] = []
            i += 1

    # a questo punto aggiungo le nuove informazioni agli slot del verbo
    for token in parsified_sentence:
        if token.dep_ != 'ROOT' and token.dep_ != 'punct':
            print(verbs[current_keys[token.head.lemma_]][token.dep_], token.dep_, token.text)

            verbs[current_keys[token.head.lemma_]][token.dep_].append(token.text)       

    break

verbs


{'prep': [], 'nsubj': [], 'advcl': [], 'xcomp': []} prep By


KeyError: 'time'

In [49]:
def generate_tree(corpus):
    parser = spacy.load('en_core_web_sm')
    verbs = {}

    for sentence in corpus['sentence']:
        sentence = sentence.replace('<s>', '').replace('</s>', '').replace('"', '').replace("'", '')[1:]
        parsified_sentence = parser(sentence)
        #spacy.displacy.render(parsified_sentence, style="dep", options={"compact": True}) # visualizza il grafo


        for token in parsified_sentence:
            #print(token.head.text, '\t--', token.dep_, '-->\t', token.text, '(', token.pos_, ')')
            if token.head.pos_ == 'VERB':
                if token.head.lemma_ not in verbs:
                    verbs[token.head.lemma_] = {}

                if token.dep_ != 'ROOT' and token.dep_ != 'punct':
                    if not token.dep_ in verbs[token.head.lemma_]:
                        verbs[token.head.lemma_][token.dep_] = []

                    verbs[token.head.lemma_][token.dep_].append(token.text)

        break

    return verbs

verbs1 = generate_tree(corpus)
print_tree(verbs1)


{'prep', 'nsubj', 'advcl', 'xcomp'}
begin
	 prep
		 By
	 nsubj
		 some
	 xcomp
		 use
	 advcl
		 seeing
-----------------------------
use
	 aux
		 to
	 dobj
		 term
	 advmod
		 positively
	 prep
		 in
-----------------------------
see
	 dobj
		 government
	 prep
		 as
-----------------------------


In [34]:
'''
per ogni verbo in verbs stampa un albero con i soggetti e gli oggetti
'''
def print_tree(verbs):
    for verb in verbs:
        print(verb)
        for dep in verbs[verb]:
            print('\t', dep)
            for word in verbs[verb][dep]:
                print('\t\t', word)
                
        print('-----------------------------')

## MAIN

In [12]:
corpus = pd.read_csv('english_wikipedia_sentence.csv')
corpus.head(5)

,sentence
0,"<s> By the time of the French Revolution some,..."
1,<s> Many workers and activists saw Bolshevik s...
2,<s> The revolutionary wave of 1917–23 saw the ...
3,<s> Italy saw the first struggles between anar...
4,<s> Anarcho-pacifism became influential in the...


In [13]:
verbs = generate_tree(corpus)

In [15]:
print_tree(verbs)

begin
	 nsubj
		 some
		 Chaucer
		 plants
	 dobj
		 trend
-----------------------------
use
	 nsubj
		 that
		 translations
		 who
	 dobj
		 term
		 equations
		 pesticides
		 yupana
		 pitch
		 hypothesis
-----------------------------
see
	 nsubj
		 workers
		 wave
		 Italy
		 philosophy
		 which
		 Italy
		 rate
		 table
		 Odysseus
		 he
		 who
		 Lincoln
		 they
		 they
		 we
		 he
		 I
		 it
		 Lord
		 Gershwin
		 Secretary
		 I
		 Rand
		 some
		 century
		 studies
		 Jung
		 animators
		 she
		 Greeks
		 we
		 he
		 that
		 he
		 1993
		 Boys
		 he
		 they
		 portions
		 Crescent
		 goats
		 Ages
		 distortions
		 crops
		 who
		 one
		 period
		 BC
		 abacus
		 crew
		 he
		 they
		 who
		 Borman
		 crew
		 that
		 year
		 historians
		 quarter
		 it
		 Swift
	 dobj
		 government
		 success
		 participation
		 struggles
		 state
		 Anarchism
		 naturism
		 anarchism
		 Anarchism
		 naturism
		 anarchism
		 Anarchism
		 naturism
		 anarchism
		 it
		 struggle
		 non
		 -
		 vio